In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# Iris

1. 붓꽃 species별로 평균, 표준편차 등 기초통계량 구하기

In [4]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [12]:
iris.groupby(iris.species).agg(["mean","std"])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

각 열에 대해 descibe()를 사용할 수 있지만 열마다 DF를 만들어야 하고 서로 비교하기 쉽지 않아 필요한 통계량만 가져올 수 있도록 agg(['함수','함수']) 사용

In [15]:
iris.groupby(iris.species).sepal_length.describe()

,count,mean,std,min,25%,50%,75%,max
species,,,,,,,,
setosa,50.0,5.006,0.352490,4.3,4.800,5.0,5.2,5.8
versicolor,50.0,5.936,0.516171,4.9,5.600,5.9,6.3,7.0
virginica,50.0,6.588,0.635880,4.9,6.225,6.5,6.9,7.9


2. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다. 이 이상치를 제거하고 각 열의 평균, 표준편차 구하기

In [23]:
def remove_outlier(column):
    q1, q3 = np.percentile(column, [25,75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr*1.5)
    upper_bound = q3 + (iqr*1.5)

    outlier_index = np.where( (column>upper_bound) | (column<lower_bound))
    print(outlier_index)
    column.drop(outlier_index, inplace = True)

In [24]:
iris.apply(remove_outlier)

(array([], dtype=int64),)


ValueError: Index data must be 1-dimensional

# Titanic

In [54]:
titanic = sns.load_dataset('titanic')
titanic.head(7)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True


In [55]:
titanic.count()

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

In [56]:
# NaN 제거
titanic.age = titanic.age.fillna(round(titanic.age.mean()))

1. 승객의 나이와 성별에 의한 카테고리 열인 category1 열을 만든다 <br>
\- 20살이 넘으면 성별을 그대로 사용한다.<br>
\- 20살 미만이면 성별에 관계없이 "Child" 라고 한다.

In [71]:
titanic['category1'] = titanic.apply(lambda r: "Child" if r.age <20 else r.age, axis=1)
titanic[['category1']]

,category1
0,22
1,38
2,26
3,35
4,35
...,...
886,27
887,Child
888,30
889,26


2. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치세요.

In [82]:
# NaN 제거 => 1번에서 함
# titanic.age = titanic.age.fillna(round(titanic.age.mean()))
titanic.head(7)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,22
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,38
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,26
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,35
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,35
5,0,3,male,30.0,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,30
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True,54


3. 성별, 선실(class)별, 출발지(embark_town)별 생존율

In [120]:
# pivot_table에 쓸만할 파라미터 2개밖에 없으면 None으로 조절 가능
sex = titanic.pivot_table('survived', 'sex',None, aggfunc="count")
sex['ratio(%)'] = round(sex['survived'] / titanic.survived.count() * 100,2)
sex

,survived,ratio(%)
sex,,
female,314,35.24
male,577,64.76


In [119]:
class_ = titanic.pivot_table('survived', 'class',None, aggfunc="count")
class_['ratio(%)'] = round(class_['survived'] / titanic.survived.count() * 100,2)
class_

,survived,ratio(%)
class,,
First,216,24.24
Second,184,20.65
Third,491,55.11


In [118]:
embark_town = titanic.pivot_table('survived', 'embark_town',None, aggfunc="count")
embark_town['ratio(%)'] = round(embark_town['survived'] / titanic.survived.count() * 100,2)
embark_town


,survived,ratio(%)
embark_town,,
Cherbourg,168,18.86
Queenstown,77,8.64
Southampton,644,72.28


4. 타이타닉호 승객을 미성년자, 청년, 중년, 장년, 노년 나이 그룹으로 나누고 각 그룹별 생존률을 구하시오.

In [132]:
bins = [1, 20, 30, 50, 70, 100]
labels = ['미성년자', '청년', '중년', '장년', '노년']
cats = pd.cut(titanic.age, bins, labels=labels)
type(cats)

pandas.core.series.Series

In [133]:
titanic['나이그룹'] = cats

In [134]:
나이그룹 = titanic.pivot_table('survived','나이그룹',None, aggfunc='count')
나이그룹['ratio(%)'] = round(나이그룹['survived'] / titanic.survived.count() * 100,2)
나이그룹


,survived,ratio(%)
나이그룹,,
미성년자,165,18.52
청년,407,45.68
중년,241,27.05
장년,59,6.62
노년,5,0.56


5. qcut 명령으로 3개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [136]:
qcat = pd.qcut(titanic.age, 3, labels=['1/3','2/3','3/3'])
titanic['Q나이그룹'] = qcat

In [137]:
Q나이그룹 = titanic.pivot_table('survived','Q나이그룹',None, aggfunc='count')
Q나이그룹['ratio(%)'] = round(Q나이그룹['survived'] / titanic.survived.count() * 100,2)
Q나이그룹

,survived,ratio(%)
Q나이그룹,,
1/3,301,33.78
2/3,304,34.12
3/3,286,32.10


# Mile Per Gallon

In [138]:
mpg = sns.load_dataset('mpg')
mpg.head(7)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
5,15.0,8,429.0,198.0,4341,10.0,70,usa,ford galaxie 500
6,14.0,8,454.0,220.0,4354,9.0,70,usa,chevrolet impala


1. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc) 추가

In [141]:
mpg['hp_per_cc'] = mpg.displacement / mpg.horsepower
mpg.head(7)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,2.361538
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,2.121212
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,2.120000
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,2.026667
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,2.157143
5,15.0,8,429.0,198.0,4341,10.0,70,usa,ford galaxie 500,2.166667
6,14.0,8,454.0,220.0,4354,9.0,70,usa,chevrolet impala,2.063636


2. name으로부터 manufacturer(제조사=> 첫번째 단어)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고 name열은 삭제

In [152]:
type(mpg.name)

pandas.core.series.Series

In [153]:
mpg.name[0]

'chevrolet chevelle malibu'

In [ ]:
mpg['manufacturer'] = mpg.apply(lambda r: r.name)